In [1]:
from numpy import array 
from keras.preprocessing.text import Tokenizer 
from keras.utils import to_categorical 
from keras.preprocessing.sequence import pad_sequences 
# from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding 

# generate a sequence from a language model 

def generate_seq(model, tokenizer, max_length, seed_text, n_words): 
    in_text = seed_text 
    # generate a fixed number of words 
    for _ in range(n_words): 
        # encode the text as integer 
        encoded = tokenizer.texts_to_sequences([in_text])[0] 
        # pre-pad sequences to a fixed length 
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre') 
        # predict probabilities for each word 
        yhat = model.predict_classes(encoded, verbose=0) 
        # map predicted word index to word 
        out_word = ''
        for word, index in tokenizer.word_index.items(): 
            if index == yhat: 
                out_word = word 
                break 
                # append to input 
        in_text += ' ' + out_word 
    return in_text 

# define the model 
def define_model(vocab_size, max_length): 
    model = Sequential() 
    model.add(Embedding(vocab_size, 10, input_length=max_length-1)) 
    model.add(LSTM(50)) 
    model.add(Dense(vocab_size, activation='softmax')) 
    # compile network 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    # summarize defined model 
    model.summary() 
#     plot_model(model, to_file='model.png', show_shapes=True) 
    return model 

Using TensorFlow backend.


In [2]:
# source text 
data = """ Jack and Jill went up the hill\n 
            To fetch a pail of water\n 
            Jack fell down and broke his crown\n 
            And Jill came tumbling after\n """ 

In [3]:
# prepare the tokenizer on the source text 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 

In [4]:
tokenizer.index_word

{1: 'and',
 2: 'jack',
 3: 'jill',
 4: 'went',
 5: 'up',
 6: 'the',
 7: 'hill',
 8: 'to',
 9: 'fetch',
 10: 'a',
 11: 'pail',
 12: 'of',
 13: 'water',
 14: 'fell',
 15: 'down',
 16: 'broke',
 17: 'his',
 18: 'crown',
 19: 'came',
 20: 'tumbling',
 21: 'after'}

In [5]:
# determine the vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 

Vocabulary Size: 22


In [6]:
# create line-based sequences 
sequences = list() 
for line in data.split('\n'): 
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)): 
        sequence = encoded[:i+1] 
        sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences)) 

Total Sequences: 21


In [7]:
# pad input sequences 
max_length = max([len(seq) for seq in sequences]) 
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre') 
print('Max Sequence Length: %d' % max_length) 

Max Sequence Length: 7


In [8]:
# split into input and output elements 
sequences = array(sequences) 
X, y = sequences[:,:-1],sequences[:,-1] 
y = to_categorical(y, num_classes=vocab_size) 

In [11]:
# define model 
model = define_model(vocab_size, max_length) 

# fit network 
model.fit(X, y, epochs=500, verbose=1) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 6, 10)             220       
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/500
21/21 [==============================] - 1s 35ms/step - loss: 3.0903 - accuracy: 0.0000e+00
Epoch 2/500
21/21 [==============================] - 0s 333us/step - loss: 3.0886 - accuracy: 0.0000e+00
Epoch 3/500
21/21 [==============================] - 0s 381us/step - loss: 3.0869 - accuracy: 0.0000e+00
Epoch 4/500
21/21 [==============================] - 0s 381us/step - loss: 3.0853 - accuracy: 0.0952
Epoch 5/500
21/21 [==============================] - 0s 333us/step - loss: 3.0835 - accuracy: 0.1905
Epoch 6/500
21/21 [==============================] - 0s 333us/step - loss: 3.0818 - accuracy: 0.1429
Epoch 7/500
21/21 [==============================] - 0s 428us/step - loss: 3.0800 - accuracy: 0.1429
Epoch 8/500
21/21 [==============================] - 0s 381us/step - loss: 3.0781 - accuracy: 0.0952
Epoch 9/500
21/21 [==============================] - 0s 429us/step - loss: 3.0762 - accuracy: 0.0952
Epoch 10/500
21/21 [==============================] - 0s 381us/step - loss: 3.07

Epoch 82/500
21/21 [==============================] - 0s 333us/step - loss: 2.1008 - accuracy: 0.3810
Epoch 83/500
21/21 [==============================] - 0s 381us/step - loss: 2.0710 - accuracy: 0.4286
Epoch 84/500
21/21 [==============================] - 0s 381us/step - loss: 2.0416 - accuracy: 0.4762
Epoch 85/500
21/21 [==============================] - 0s 333us/step - loss: 2.0125 - accuracy: 0.4762
Epoch 86/500
21/21 [==============================] - 0s 333us/step - loss: 1.9839 - accuracy: 0.4762
Epoch 87/500
21/21 [==============================] - 0s 381us/step - loss: 1.9562 - accuracy: 0.3810
Epoch 88/500
21/21 [==============================] - 0s 381us/step - loss: 1.9290 - accuracy: 0.3810
Epoch 89/500
21/21 [==============================] - 0s 333us/step - loss: 1.9026 - accuracy: 0.4286
Epoch 90/500
21/21 [==============================] - 0s 381us/step - loss: 1.8769 - accuracy: 0.4286
Epoch 91/500
21/21 [==============================] - 0s 381us/step - loss: 1.8516

Epoch 162/500
21/21 [==============================] - 0s 381us/step - loss: 0.8287 - accuracy: 0.8095
Epoch 163/500
21/21 [==============================] - 0s 333us/step - loss: 0.8218 - accuracy: 0.8095
Epoch 164/500
21/21 [==============================] - 0s 333us/step - loss: 0.8151 - accuracy: 0.8095
Epoch 165/500
21/21 [==============================] - 0s 381us/step - loss: 0.8083 - accuracy: 0.8095
Epoch 166/500
21/21 [==============================] - 0s 428us/step - loss: 0.8013 - accuracy: 0.8095
Epoch 167/500
21/21 [==============================] - 0s 333us/step - loss: 0.7942 - accuracy: 0.8095
Epoch 168/500
21/21 [==============================] - 0s 333us/step - loss: 0.7873 - accuracy: 0.8095
Epoch 169/500
21/21 [==============================] - 0s 333us/step - loss: 0.7807 - accuracy: 0.8095
Epoch 170/500
21/21 [==============================] - 0s 381us/step - loss: 0.7744 - accuracy: 0.8095
Epoch 171/500
21/21 [==============================] - 0s 381us/step - lo

Epoch 242/500
21/21 [==============================] - 0s 333us/step - loss: 0.4552 - accuracy: 0.8571
Epoch 243/500
21/21 [==============================] - 0s 333us/step - loss: 0.4522 - accuracy: 0.8571
Epoch 244/500
21/21 [==============================] - 0s 333us/step - loss: 0.4493 - accuracy: 0.8571
Epoch 245/500
21/21 [==============================] - 0s 333us/step - loss: 0.4464 - accuracy: 0.8571
Epoch 246/500
21/21 [==============================] - 0s 381us/step - loss: 0.4435 - accuracy: 0.8571
Epoch 247/500
21/21 [==============================] - 0s 333us/step - loss: 0.4407 - accuracy: 0.8571
Epoch 248/500
21/21 [==============================] - 0s 381us/step - loss: 0.4379 - accuracy: 0.8571
Epoch 249/500
21/21 [==============================] - 0s 381us/step - loss: 0.4351 - accuracy: 0.8571
Epoch 250/500
21/21 [==============================] - 0s 333us/step - loss: 0.4323 - accuracy: 0.8571
Epoch 251/500
21/21 [==============================] - 0s 333us/step - lo

Epoch 322/500
21/21 [==============================] - 0s 429us/step - loss: 0.2648 - accuracy: 0.9524
Epoch 323/500
21/21 [==============================] - 0s 428us/step - loss: 0.2628 - accuracy: 0.9524
Epoch 324/500
21/21 [==============================] - 0s 429us/step - loss: 0.2609 - accuracy: 0.9524
Epoch 325/500
21/21 [==============================] - 0s 476us/step - loss: 0.2589 - accuracy: 0.9524
Epoch 326/500
21/21 [==============================] - 0s 381us/step - loss: 0.2570 - accuracy: 0.9524
Epoch 327/500
21/21 [==============================] - 0s 381us/step - loss: 0.2552 - accuracy: 0.9524
Epoch 328/500
21/21 [==============================] - 0s 333us/step - loss: 0.2533 - accuracy: 0.9524
Epoch 329/500
21/21 [==============================] - 0s 334us/step - loss: 0.2514 - accuracy: 0.9524
Epoch 330/500
21/21 [==============================] - 0s 381us/step - loss: 0.2496 - accuracy: 0.9524
Epoch 331/500
21/21 [==============================] - 0s 381us/step - lo

Epoch 402/500
21/21 [==============================] - 0s 381us/step - loss: 0.1537 - accuracy: 0.9524
Epoch 403/500
21/21 [==============================] - 0s 286us/step - loss: 0.1528 - accuracy: 0.9524
Epoch 404/500
21/21 [==============================] - 0s 333us/step - loss: 0.1520 - accuracy: 0.9524
Epoch 405/500
21/21 [==============================] - 0s 333us/step - loss: 0.1511 - accuracy: 0.9524
Epoch 406/500
21/21 [==============================] - 0s 381us/step - loss: 0.1503 - accuracy: 0.9524
Epoch 407/500
21/21 [==============================] - 0s 333us/step - loss: 0.1495 - accuracy: 0.9524
Epoch 408/500
21/21 [==============================] - 0s 333us/step - loss: 0.1487 - accuracy: 0.9524
Epoch 409/500
21/21 [==============================] - 0s 333us/step - loss: 0.1479 - accuracy: 0.9524
Epoch 410/500
21/21 [==============================] - 0s 333us/step - loss: 0.1471 - accuracy: 0.9524
Epoch 411/500
21/21 [==============================] - 0s 429us/step - lo

Epoch 482/500
21/21 [==============================] - 0s 333us/step - loss: 0.1102 - accuracy: 0.9524
Epoch 483/500
21/21 [==============================] - 0s 333us/step - loss: 0.1098 - accuracy: 0.9524
Epoch 484/500
21/21 [==============================] - 0s 333us/step - loss: 0.1095 - accuracy: 0.9524
Epoch 485/500
21/21 [==============================] - 0s 381us/step - loss: 0.1092 - accuracy: 0.9524
Epoch 486/500
21/21 [==============================] - 0s 381us/step - loss: 0.1089 - accuracy: 0.9524
Epoch 487/500
21/21 [==============================] - 0s 381us/step - loss: 0.1086 - accuracy: 0.9524
Epoch 488/500
21/21 [==============================] - 0s 286us/step - loss: 0.1083 - accuracy: 0.9524
Epoch 489/500
21/21 [==============================] - 0s 381us/step - loss: 0.1080 - accuracy: 0.9524
Epoch 490/500
21/21 [==============================] - 0s 333us/step - loss: 0.1077 - accuracy: 0.9524
Epoch 491/500
21/21 [==============================] - 0s 381us/step - lo

In [13]:
# evaluate model 
print(generate_seq(model, tokenizer, max_length-1, 'cherry', 3)) 
print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4))

cherry and jill came
Jill jill came tumbling after
